In [1]:
import os
from dotenv import load_dotenv
load_dotenv('key.env')  

key_string = os.getenv('open_ai_API_Key')

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import StructuredTool, Tool
from langchain_openai import ChatOpenAI

from langchain.tools import tool

/Users/gunik/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3526: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
class Extract_toolArgs(BaseModel):
    a: str = Field(description="Company name")
    b: str = Field(description="item")
    c: str = Field(description="Year_1")
    d: str = Field(description="Year_2")


In [4]:
years  = [2018, 2019, 2020, 2021, 2022, 2023, 2024]

In [13]:
import json
@tool(args_schema=Extract_toolArgs)
def Extract_Tool(a: str, b: str, c:str, d:str) -> list:
    """extracts specific data for a company from Year_1 to Year_2."""
    with open("data_soo.json","r") as json_file:
        data = json.load(json_file)
    a = a.upper()
    json_data = next(item for item in data[a] if item["Item"] == b)
    cash_data = []
    if int(c) > int(d): c, d = d, c
    for year in years:
        if year >= int(c) and year <= int(d):
            cash_data.append(json_data.get(str(year)))
    # sales_year_1 = json_data.get(c)
    # sales_year_2 = json_data.get(d)
    if len(cash_data) == 0:
        raise ValueError(f"Data for the years {c} to {d} is not available.")
    
    return cash_data

tools = [Extract_Tool]


In [14]:
# class CalculateKpIs(BaseModel):
#     a: str = Field(description="Company name")
#     b: str = Field(description="item")
#     c: list = Field(description="list of revenue data points")

# @tool(args_schema=CalculateKpIs)
# def CalculateKpIs(a: str, b: str, c:list) -> list:
#     """calculates kpis according to company and item name"""

In [15]:
llm = ChatOpenAI(model="gpt-4o", openai_api_key=key_string)

In [16]:
prompt = hub.pull("hwchase17/openai-tools-agent")

/Users/gunik/anaconda3/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
agent = create_tool_calling_agent(
    llm=llm,  # Language model to use
    tools=tools,  # List of tools available to the agent
    prompt=prompt,  # Prompt template to guide the agent's responses
)

In [18]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,  # The agent to execute
    tools=tools,  # List of tools available to the agent
    verbose=True,  # Enable verbose logging
    handle_parsing_errors=True,  # Handle parsing errors gracefully
)

In [21]:
response = agent_executor.invoke({"input": "calculate growth in Net sales of apple from 2021 to 2024"})



> Entering new AgentExecutor chain...

Invoking: `Extract_Tool` with `{'a': 'Apple', 'b': 'Net sales', 'c': '2021', 'd': '2024'}`


[365817, 394328, 383285, 391035]To calculate the growth in net sales of Apple from 2021 to 2024, we can use the following formula for growth percentage:

\[
\text{Growth Percentage} = \left(\frac{\text{Net Sales in 2024} - \text{Net Sales in 2021}}{\text{Net Sales in 2021}}\right) \times 100
\]

Given the net sales:
- 2021: $365,817 million
- 2024: $391,035 million

Plugging in the values:

\[
\text{Growth Percentage} = \left(\frac{391035 - 365817}{365817}\right) \times 100 \approx 6.88\%
\]

The net sales of Apple grew by approximately 6.88% from 2021 to 2024.

> Finished chain.


In [20]:
print(response)

{'input': 'calculate Net sales of apple from 2021 to 2024', 'output': 'The Net Sales of Apple from 2021 to 2024 are as follows:\n\n- 2021: $365,817 million\n- 2022: $394,328 million\n- 2023: $383,285 million\n- 2024: $391,035 million'}
